# FCIQMC (Full Configuration Interaction Quantum Monte Carlo)

論文 [Fermion Monte Carlo without fixed nodes: A game of life, death, and
annihilation in Slater determinant space](https://2024.sci-hub.st/1534/f26924b07d1005f6f9a7be1b525feec7/booth2009.pdf) を読みPythonでの実装を試みる。

## 準備

```
$ conda install -c conda-forge pyscf
```

Geminiで生成したコード

In [13]:
import numpy as np
from pyscf import gto, scf, fci, ao2mo
import random

class SimpleFCIQMC:
    def __init__(self, mol, n_walkers=1000, time_step=0.01, steps=5000):
        self.mol = mol
        self.N_w_target = n_walkers  # 目標ウォーカー数
        self.dt = time_step          # タイムステップ (tau)
        self.steps = steps           # 反復回数
        self.shift = 0.0             # エネルギーシフト (S)
        self.damping = 0.1           # シフト更新の減衰パラメータ
        
        # ハミルトニアン行列の準備
        self.H_mat, self.e_hf, self.norb, self.nelec = self._build_hamiltonian()
        
        # 基底状態（Hartree-Fock解）のインデックスを特定
        # PySCFのFCIでは通常、最初の行列要素がHF状態に対応します
        self.ref_det_idx = 0 
        
        # ウォーカーの初期化 {det_index: signed_population}
        # 符号付き整数で管理（+1, -1など）
        self.walkers = {self.ref_det_idx: self.N_w_target}
        
        # 統計用
        self.energy_hist = []

    def _build_hamiltonian(self):
        """PySCFを使ってFCIハミルトニアン行列全体を生成する"""
        # 1. RHF計算
        mf = scf.RHF(self.mol).run(verbose=0)
        h1 = mf.get_hcore()
        h2 = mf._eri
        
        # 2. 分子軌道(MO)積分へ変換
        norb = mf.mo_coeff.shape[1]
        nelec = self.mol.nelectron
        h1_mo = np.dot(mf.mo_coeff.T, np.dot(h1, mf.mo_coeff))
        h2_mo = ao2mo.kernel(self.mol, mf.mo_coeff)
        
        # 3. 全CIハミルトニアン行列の生成
        # 注意: H2/STO-3Gのような小規模系でのみ可能です。
        # 大規模系ではオンザフライで行列要素を計算する必要があります。
        cisolver = fci.direct_spin1.FCI(self.mol)
        h2_mo_restore = ao2mo.restore(1, h2_mo, norb)
        H_mat = fci.direct_spin1.pspace(h1_mo, h2_mo_restore, norb, nelec)[1]
        
        return H_mat, mf.e_tot, norb, nelec

    def run(self):
        print(f"Starting FCIQMC for {self.mol.atom}...")
        print(f"Hartree-Fock Energy: {self.e_hf:.6f} Ha")
        
        diag_H = np.diag(self.H_mat)
        
        # シフトの初期値をHFエネルギー付近に設定（核反発エネルギー込み）
        # H_matは電子ハミルトニアンなので、核反発を加える必要があるが、
        # ここではダイナミクスのためにH_matの基底期待値に合わせる
        self.shift = diag_H[self.ref_det_idx]
        print('shfit: ', self.shift)

        for step in range(self.steps):
            new_walkers = {}
            
            # --- 1. Spawning & Death/Cloning Step ---
            # 現在の全ウォーカーに対してループ
            current_dets = list(self.walkers.keys())
            
            for i in current_dets:
                n_i = self.walkers[i] # 決定基i上のウォーカー数（符号付き）
                if n_i == 0: continue
                
                # A. Diagonal Step (Death/Cloning)
                # Pd = exp(-dt * (H_ii - S)) - 1  ~ -dt * (H_ii - S)
                # ここでは単純な一次近似を使用
                h_ii = diag_H[i]
                prob_death = self.dt * (h_ii - self.shift)
                # 既存のウォーカー数 n_i を増減させる
                # 確率的に床関数/天井関数を使って整数化
                change = -(n_i * prob_death)
                
                # 確率的整数化 (Stochastic rounding)
                change_int = int(change)
                if random.random() < abs(change - change_int):
                    change_int += int(np.sign(change))
                
                # 生き残ったウォーカーを一時リストに追加
                if i not in new_walkers: new_walkers[i] = 0
                new_walkers[i] += n_i + change_int

                # B. Off-Diagonal Step (Spawning)
                # 決定基 i から j へのスポーニング
                # 通常は接続された決定基をランダムサンプリングするが、
                # ここでは行列が小さいため、接続している全 j を走査（簡単のため）
                row = self.H_mat[i]
                # 非ゼロ要素のインデックスを取得（自分自身を除く）
                connected_dets = np.where(row != 0)[0]
                
                for j in connected_dets:
                    if i == j: continue
                    h_ij = row[j]
                    
                    # Spawning probability: Ps = dt * |H_ij|
                    # 生成される数: sign(walker) * sign(-H_ij)
                    prob_spawn = self.dt * abs(h_ij)
                    n_spawn = abs(n_i) * prob_spawn
                    
                    # 確率的整数化
                    n_spawn_int = int(n_spawn)
                    if random.random() < (n_spawn - n_spawn_int):
                        n_spawn_int += 1
                    
                    if n_spawn_int > 0:
                        # 符号の決定 (反対称性を考慮: H_ij < 0 なら同符号、H_ij > 0 なら異符号)
                        child_sign = np.sign(n_i) * -np.sign(h_ij)
                        
                        if j not in new_walkers: new_walkers[j] = 0
                        new_walkers[j] += int(child_sign * n_spawn_int)

            # --- 2. Annihilation Step ---
            # new_walkers辞書への加算処理ですでに、同じ決定基上の
            # 正負のウォーカーは相殺（Annihilation）されている。
            # 数が0になった決定基を削除
            self.walkers = {k: v for k, v in new_walkers.items() if v != 0}
            
            # --- 3. Shift Update ---
            total_walkers = sum(abs(w) for w in self.walkers.values())
            if step > 500: # 初期緩和後
                self.shift -= (self.damping / self.dt) * np.log(total_walkers / self.N_w_target)
            
            # --- 4. Energy Estimation (Projected Energy) ---
            # E = <D0|H|Psi> / <D0|Psi>
            # HF決定基(D0)上のウォーカー数と、D0からつながるH要素で計算
            
            num_ref = self.walkers.get(self.ref_det_idx, 0)
            if num_ref != 0:
                # Hの対角成分（D0のエネルギー）
                energy_proj = diag_H[self.ref_det_idx]
                
                # Hの非対角成分からの寄与 (sum_{j!=0} H_0j * N_j) / N_0
                off_diag_sum = 0
                row0 = self.H_mat[self.ref_det_idx]
                for det_idx, n_w in self.walkers.items():
                    if det_idx != self.ref_det_idx:
                        off_diag_sum += row0[det_idx] * n_w
                
                energy_proj += off_diag_sum / num_ref
                
                # 核反発エネルギーを足して全エネルギーへ
                total_energy = energy_proj + self.mol.energy_nuc()
                self.energy_hist.append(total_energy)
            
            if step % 500 == 0:
                print(f"Step {step}: Walkers={total_walkers}, Shift={self.shift:.5f}, E_proj={self.energy_hist[-1] if self.energy_hist else 0:.6f}")

        return np.mean(self.energy_hist[-1000:]) # 最後の1000ステップの平均

# --- 実行 ---
# H2分子の定義 (0.74 Angstrom)
mol = gto.M(atom='H 0 0 0; H 0 0 0.74', basis='sto-3g', verbose=0)

# FCIQMCの実行
fci_qmc = SimpleFCIQMC(mol, n_walkers=2000, time_step=0.005, steps=10000)
fci_energy = fci_qmc.run()

# 正解値（厳密な対角化）との比較
print("-" * 30)
print(f"Final FCIQMC Energy: {fci_energy:.6f} Ha")

# PySCFの厳密解(FCI)
cisolver = fci.FCI(mol, scf.RHF(mol).run(verbose=0))
print(f"Exact FCI Energy   : {cisolver.kernel()[0]:.6f} Ha")

Starting FCIQMC for H 0 0 0; H 0 0 0.74...
Hartree-Fock Energy: -1.116759 Ha
shfit:  -1.831863646477506
Step 0: Walkers=2002, Shift=-1.83186, E_proj=-1.116941
Step 500: Walkers=2308, Shift=-1.83186, E_proj=-1.136913
Step 1000: Walkers=2041, Shift=-10.25552, E_proj=-1.137212
Step 1500: Walkers=1764, Shift=-0.59021, E_proj=-1.136590
Step 2000: Walkers=2026, Shift=6.12358, E_proj=-1.137159
Step 2500: Walkers=2275, Shift=-4.41973, E_proj=-1.136550
Step 3000: Walkers=1912, Shift=-8.74192, E_proj=-1.137222
Step 3500: Walkers=1782, Shift=3.25968, E_proj=-1.137372
Step 4000: Walkers=2158, Shift=4.35684, E_proj=-1.137226
Step 4500: Walkers=2220, Shift=-8.57565, E_proj=-1.136291
Step 5000: Walkers=1791, Shift=-6.14564, E_proj=-1.137382
Step 5500: Walkers=1859, Shift=5.89716, E_proj=-1.138239
Step 6000: Walkers=2260, Shift=1.48258, E_proj=-1.137092
Step 6500: Walkers=2092, Shift=-10.21172, E_proj=-1.138055
Step 7000: Walkers=1739, Shift=-1.50645, E_proj=-1.137035
Step 7500: Walkers=1998, Shift=7.

TypeError: 'RHF' object is not subscriptable

In [26]:
import numpy as np
from pyscf import gto, scf, fci

# ===============================
# 1. Build H2 STO-3G Hamiltonian
# ===============================
mol = gto.Mole()
mol.atom = "H 0 0 0; H 0 0 0.74"
mol.basis = "sto-3g"
mol.build()

mf = scf.RHF(mol).run()

cisolver = fci.FCI(mf)
H_mat = cisolver.kernel(return_matrix=True)[1]

dim = H_mat.shape[0]
print("Number of determinants:", dim)
print("FCI matrix:\n", H_mat)

# ==================================
# 2. FCIQMC Simulation (Booth 2009)
# ==================================
n_steps = 10
delta_tau = 0.003
target_population = 1000

# Shift parameter (population control)
shift = 0.0
shift_update_rate = 0.01  # α in the paper

# Walker population vector (signed integer)
walkers = np.zeros(dim, dtype=float)

# Start with random walker on HF determinant (index 0)
walkers[0] = 10

population_history = []
energy_estimates = []

rng = np.random.default_rng()

for step in range(n_steps):

    new_walkers = np.zeros(dim)

    for i in range(dim):
        n_i = int(abs(walkers[i]))
        sign_i = np.sign(walkers[i])
        if n_i == 0:
            continue

        # --------- Spawning (off-diagonal hopping) ---------
        for _ in range(n_i):
            # randomly choose connected determinant j
            j = rng.integers(0, dim)

            if j != i and H_mat[j, i] != 0:
                prob = abs(H_mat[j, i]) * delta_tau

                if rng.random() < prob:
                    sign = np.sign(H_mat[j, i]) * sign_i
                    new_walkers[j] += sign

        # --------- Death/Cloning (diagonal contribution) ---------
        diag_term = (H_mat[i, i] - shift)
        death_prob = diag_term * delta_tau

        for _ in range(n_i):
            if rng.random() < death_prob:
                walkers[i] -= sign_i
            else:
                walkers[i] += sign_i

    walkers += new_walkers

    # --------- Annihilation ---------
    walkers = np.round(walkers)  # clean fractional walkers
    for i in range(dim):
        if abs(walkers[i]) < 1:
            walkers[i] = 0

    # --------- Population Control ---------
    total_pop = np.sum(abs(walkers))
    shift += shift_update_rate * np.log(total_pop / target_population)

    # --------- Energy Estimator (Projected) ---------
    if walkers[0] != 0:
        E_proj = H_mat[0] @ walkers / walkers[0]
        energy_estimates.append(E_proj)

    population_history.append(total_pop)

# ================================
# Output Results
# ================================
print("\n==== FCIQMC Result ====")
print("Mean projected energy:", np.mean(energy_estimates[-2000:]))
print("Exact FCI energy:", cisolver.kernel()[0])


converged SCF energy = -1.11675930739642
Number of determinants: 2
FCI matrix:
 [[ 9.93646755e-01  1.38777878e-16]
 [ 8.45112964e-17 -1.12543887e-01]]

==== FCIQMC Result ====
Mean projected energy: 0.9936467548998384
Exact FCI energy: -1.137283834488502


In [44]:
import numpy as np
from pyscf import gto, scf, fci

# ===============================
# 1. Build H2 STO-3G Hamiltonian
# ===============================
mol = gto.Mole()
mol.atom = "H 0 0 0; H 0 0 0.74"
mol.basis = "sto-3g"
mol.build()

mf = scf.RHF(mol).run()

cisolver = fci.FCI(mf)
K = cisolver.kernel(return_matrix=True)[1]

dim = K.shape[0]
print("Number of determinants:", dim)
print("FCI matrix:\n", K)

# ==================================
# 2. FCIQMC Simulation (Booth 2009)
# ==================================
n_steps = 20
delta_tau = 0.003
target_population = 1000

# Shift parameter (population control)
# shift = 0.0
shift = -1.8  # 適当な負の値にしておく
shift_update_rate = 0.01  # α in the paper TODO: たぶん違う

# Walker population vector (signed integer)
walkers = {i: [] for i in range(dim)}
walkers[0] = [+1] # 初期のwalkerは一つだけ

# TODO: walkersのデータ構造を変えたのでそれに合わせて以降の実装を整える

# population_history = []
# energy_estimates = []

rng = np.random.default_rng()

for step in range(n_steps):

    # new_walkers = np.zeros(dim)
    
    for i in range(dim):
        n_i = int(abs(walkers[i]))
        sign_i = np.sign(walkers[i])
        if n_i == 0:
            continue

        # --------- Spawning (off-diagonal hopping) ---------
        for _ in range(n_i):
            # randomly choose connected determinant j
            # 論文には遷移先候補をランダムに選ぶとは書いていない
            # j = rng.integers(0, dim)
            # Kの非対角要素、かつ0ではない要素からインデックスjを得る
            targets = [j for j in range(dim) if K[j,i] != 0 and j != i]
            j = rng.choice(targets)

            # TODO: 遷移確率の分母が実装されていない
            prob = abs(K[j, i]) * delta_tau
            # print('prob: ', prob)

            if rng.random() < prob:
                sign = np.sign(K[j, i]) * sign_i
                new_walkers[j] += sign

            # print('new_walkers: ', new_walkers)

        # --------- Death/Cloning (diagonal contribution) ---------
    #     diag_term = (K[i, i] - shift)
    #     death_prob = diag_term * delta_tau
    #     print('death_prob: ', death_prob)
            
    #     for _ in range(n_i):
    #         if rng.random() < np.abs(death_prob):
    #             if death_prob > 0:
    #                 walkers[i] -= sign_i
    #             else:
    #                 walkers[i] += sign_i

    # walkers += new_walkers
    # print('walkers: ', walkers)

    # # --------- Annihilation ---------
    # walkers = np.round(walkers)  # clean fractional walkers
    # for i in range(dim):
    #     if abs(walkers[i]) < 1:
    #         walkers[i] = 0

    # # --------- Population Control ---------
    # total_pop = np.sum(abs(walkers))
    # shift += shift_update_rate * np.log(total_pop / target_population)

    # # --------- Energy Estimator (Projected) ---------
    # if walkers[0] != 0:
    #     E_proj = K[0] @ walkers / walkers[0]
    #     energy_estimates.append(E_proj)

    # population_history.append(total_pop)

# ================================
# Output Results
# ================================
# print("\n==== FCIQMC Result ====")
# print("Mean projected energy:", np.mean(energy_estimates[-2000:]))
# print("Exact FCI energy:", cisolver.kernel()[0])


converged SCF energy = -1.11675930739642
Number of determinants: 2
FCI matrix:
 [[ 9.93646755e-01  1.38777878e-16]
 [ 8.45112964e-17 -1.12543887e-01]]
